In [1]:
import sys
import os

# Determine the absolute path to the root directory (assuming the script is one level down)
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the root directory to sys.path if it’s not already there
if root_path not in sys.path:
    sys.path.insert(0, root_path)


In [5]:
import sqlite3

def list_tables(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Execute a query to get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Close the connection
    conn.close()

    # Return a list of table names (each item in 'tables' is a tuple)
    return [table[0] for table in tables]

In [6]:
list_tables('../data/trading_system.db')

['daily_prices',
 'company_info',
 'balance_sheet',
 'income_statement',
 'cash_flow']

In [7]:
from strategies.momentum.awesome_oscillator_strat import AwesomeOscillatorStrategy

In [5]:
from src.database.config import DatabaseConfig
from src.database.engine import create_db_engine
db_config = DatabaseConfig.default()

In [15]:
# Instantiate the strategy, passing the db_config
ao_strategy = AwesomeOscillatorStrategy(db_config, params={'short_period': 5, 'long_period': 34})

# Now you can use ao_strategy to generate signals, and it will use the database engine
signals_df = ao_strategy.generate_signals(ticker='RELIANCE.BO')

In [16]:
signals_df

,close,ao,signal,signal_strength
date,,,,
2025-02-07 00:00:00.000000,1266.849976,22.861916,0,0.0
2025-02-07 00:00:00.000000,1266.849976,22.438089,0,0.0
2025-02-07 00:00:00.000000,1266.849976,18.990734,0,0.0
2025-02-07 00:00:00.000000,1266.849976,16.163231,0,0.0


In [2]:
import pandas as pd

tickers = pd.read_excel("../data/ticker.xlsx")
tickers = tickers[~tickers["Security Name"].duplicated()]
tickers.reset_index(drop=True, inplace=True)

def add_ticker_suffix(x):
    if x["Exchange"]=="BSE":
        return x["Security Name"] + ".BO"
    else:
        return x["Security Name"] + ".NS"

tickers["Ticker"] = tickers.apply(add_ticker_suffix, axis = 1)
all_tickers = tickers["Ticker"].tolist()

In [37]:
# Now you can use ao_strategy to generate signals, and it will use the database engine
signals_df = ao_strategy.generate_signals(ticker=all_tickers[200])
signals_df

,close,ao,signal,signal_strength
date,,,,
2025-02-04 00:00:00.000000,3716.600098,-290.215871,0,0.0
2025-02-05 00:00:00.000000,3679.850098,-256.966467,0,0.0
2025-02-06 00:00:00.000000,3704.399902,-214.347347,0,0.0
2025-02-07 00:00:00.000000,3701.300049,-168.933827,0,0.0


In [22]:
from src.database.utils import query_to_dataframe


In [31]:
ticker = all_tickers[500]
a = query_to_dataframe(f"SELECT * FROM daily_prices WHERE ticker = '{ticker}'", db_path='../data/trading_system.db')


In [32]:
a

,index,date,open,high,low,close,volume,Dividends,Stock Splits,ticker,updated_at,data_source
0,0,2019-04-16 00:00:00.000000,611.682461,645.310490,608.783492,632.748291,22985293,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
1,1,2019-04-18 00:00:00.000000,634.777646,642.508230,593.322362,621.877197,5415019,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
2,2,2019-04-22 00:00:00.000000,621.345810,624.051503,604.193573,606.947571,1059760,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
3,3,2019-04-23 00:00:00.000000,606.850875,626.177336,606.850875,612.745422,1208860,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
4,4,2019-04-24 00:00:00.000000,615.064622,618.446752,607.092457,616.030945,1016453,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
...,...,...,...,...,...,...,...,...,...,...,...,...
1433,1433,2025-02-03 00:00:00.000000,5834.450195,5919.799805,5745.049805,5857.750000,405813,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
1434,1434,2025-02-04 00:00:00.000000,5866.700195,5987.000000,5860.049805,5963.899902,326464,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
1435,1435,2025-02-05 00:00:00.000000,6039.000000,6092.250000,5964.549805,6037.299805,341372,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
1436,1436,2025-02-06 00:00:00.000000,6058.399902,6062.600098,5850.000000,5859.000000,191057,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance


In [30]:
a['date'].duplicated().sum()

np.int64(4)

In [40]:
for i in range(0, len(all_tickers)):
    ticker = all_tickers[i]
    a = ao_strategy.generate_signals(ticker)
    a['ticker'] = ticker
    if i == 0:
        df = a
    else:
        df = pd.concat([df, a])

Insufficient data for analysis
Insufficient data for AO calculation for KAMAHOLD.BO. Required 36 records.


In [41]:
df

,close,ao,signal,signal_strength,ticker
2025-02-07 00:00:00.000000,1732.099976,31.116458,0.0,0.0,HDFCBANK.BO
2025-02-07 00:00:00.000000,1732.099976,41.226611,0.0,0.0,HDFCBANK.BO
2025-02-07 00:00:00.000000,1732.099976,45.677204,0.0,0.0,HDFCBANK.BO
2025-02-07 00:00:00.000000,1732.099976,49.157350,0.0,0.0,HDFCBANK.BO
2025-02-07 00:00:00.000000,4029.350098,-8.770191,0.0,0.0,TCS.BO
...,...,...,...,...,...
2025-02-07 00:00:00.000000,1526.400024,-93.070141,0.0,0.0,RAYMOND.NS
2025-02-04 00:00:00.000000,1290.500000,-319.025155,0.0,0.0,CDSL.NS
2025-02-05 00:00:00.000000,1348.849976,-293.114421,0.0,0.0,CDSL.NS
2025-02-06 00:00:00.000000,1328.050049,-266.467363,0.0,0.0,CDSL.NS


In [44]:
df['signal_strength'].describe()

count    2014.000000
mean        0.419074
std         3.617582
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        84.697567
Name: signal_strength, dtype: float64

In [51]:
from src.strategies.momentum.cci_oscillator_strat import CCIStrategy

In [64]:
cci_strategy = CCIStrategy(db_config)

In [65]:
for i in range(0, len(all_tickers)):
    ticker = all_tickers[i]
    a = cci_strategy.generate_signals(ticker)
    a['ticker'] = ticker
    if i == 0:
        df = a
    else:
        df = pd.concat([df, a])

Insufficient data for analysis


In [69]:
df['cci'].describe()

count    117736.000000
mean         -0.328490
std         111.271282
min        -663.300970
25%         -85.861790
50%          -2.360951
75%          83.829841
max         544.767753
Name: cci, dtype: float64

In [73]:
df['signal_strength'].describe()

count    127449.000000
mean          1.950381
std          12.735737
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max         570.129538
Name: signal_strength, dtype: float64

In [3]:
from src.strategies.momentum.coppock_curve_strat import CoppockCurveStrategy

In [6]:
coppock_strategy = CoppockCurveStrategy(db_config)

In [ ]:
for i in range(0, len(all_tickers)):
    ticker = all_tickers[i]
    print(i)
    a = coppock_strategy.generate_signals(ticker)
    a['ticker'] = ticker
    if i == 0:
        df = a
    else:
        df = pd.concat([df, a])

In [98]:
print(f"--- Entering _generate_signal_df ---", flush=True)


--- Entering _generate_signal_df ---


In [11]:
a = coppock_strategy.generate_signals(all_tickers[0])

/home/bhvignesh/trading_system/src/strategies/momentum/coppock_curve_strat.py:136: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return mask.fillna(0).astype(bool).shift().fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/coppock_curve_strat.py:136: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return mask.fillna(0).astype(bool).shift().fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/coppock_curve_strat.py:122: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future versi

In [10]:
a


,close,cc,signal,position,strength
date,,,,,
